# Simulation 11: Seq Greedy v. Naive Timestep Sweep

TJ Kim
1/10/21

Run the eleventh simulation with the following considerations.

- 10 Servers, 15 Users, 30 TS
- Use ONE-mobility
- Job lengths: vr = 30, Assist = 30, AR = 30
- We share the same users and jobs between simulations

Resources: Normal
- All jobs are run with standard Quantity
- 14 level 1, 5 level 2, 1 level 3

In [1]:
from IPython.core.debugger import set_trace

# Import Generic Classes
import numpy as np
import copy
import pickle
import random

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/classes")
sys.path.append(os.path.pardir+"/solvers")
sys.path.append(os.path.pardir+"/one_mobility")
sys.path.append(os.path.curdir + "/sim_store/sim11")


from Server import *
from User import *
from Link import *
from Job import *
from Migration_Plans import *
from Cost_Recordings import *

from sim11_setting import *

# Import Solver Classes
from Optim_PlanGenerator import *
from SeqGreedy_PlanGenerator import *
from Myopic_PlanGenerator import *
from Naive_PlanGenerator import *
from Cloud_PlanGenerator import *

### Run Simulation across 10 trials for the groups

Heuristics - Sequential Greedy 

Baseline - Naive

In [2]:
num_trials = 10
SG_cost_stores = []
Naive_cost_stores = []
for i in range(num_trials):
    
    print("Trial Number:",i)

    users, servers, links, jobs, sim_param = simulation_setting()
    jobs2 = copy.deepcopy(jobs)
    for j in range(len(jobs2)):
        jobs2[j].info_from_usr(users[j],refresh_rate,refresh)
    
    # Make SG Prob
    # Set Refresh to false for SG
    refresh_rate = [0,0]
    refresh = False

    jobs2 = copy.deepcopy(jobs)
    for j in range(len(jobs2)):
        jobs2[j].info_from_usr(users[j],refresh_rate,refresh)
    
    print("SG prob...")
    SG_prob = SeqGreedy_PlanGenerator(users, servers, links, jobs2, sim_param)
    SG_plan = Migration_Plans(users,jobs2,sim_param)
    SG_plan.from_seq_greedy(SG_prob)
    SG_cost_stores += [Cost_Recordings(SG_plan)]
    print("done!")
    
    # Make Naive Prob
    print("Naive prob...")
    Naive_prob = Naive_PlanGenerator(users, servers, links, jobs, sim_param)
    Naive_plan = Migration_Plans(users,jobs,sim_param)
    Naive_plan.from_seq_greedy(Naive_prob)
    Naive_cost_stores += [Cost_Recordings(Naive_plan)]
    print("done!")
    

Trial Number: 0


NameError: name 'refresh_rate' is not defined

### Pickle the simulation results

In [ ]:
import pickle

cost_stores_dic = {}
cost_stores_dic["SG"] = SG_cost_stores
cost_stores_dic["Naive"] = Naive_cost_stores
pickle.dump(cost_stores_dic, open(os.path.curdir + "/sim_store/sim11/cost_store_50ts.p", "wb" ) )

### Plot Results
Plot cost per timestep averaged over all trials.

In [ ]:
SG_cost_stores[0].all_cost_instance['total_cost'].shape

In [ ]:
SG_cost_per_time = np.zeros(sim_param.time_steps)
Naive_cost_per_time = np.zeros(sim_param.time_steps)

for i in range(num_trials):
    SG_cost_per_time += np.divide(SG_cost_stores[i].all_cost_instance['total_cost'],num_trials)
    Naive_cost_per_time += np.divide(Naive_cost_stores[i].all_cost_instance['total_cost'],num_trials)

In [ ]:
SG_cost_per_time

In [ ]:
Naive_cost_per_time

In [ ]:
# Plot the plot
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# Set Font Size Limitations
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)
figure(num=None, figsize=(5.5, 2.5), dpi=80, facecolor='w', edgecolor='k')

x_ax = np.arange(30)

plt.plot(x_ax, SG_cost_per_time, 'r--', label = 'Seq Greedy')
plt.plot(x_ax, Naive_cost_per_time, 'g-x', label = 'Naive')

plt.legend(loc = 'lower right')

plt.title('Performance and Look Ahead Window')
plt.xlabel('Time Step')
plt.ylabel('Cost Per Time Step (Dollars)')